# 基板上PVDF分解解析ノートブック

このノートブックは、Al2O3、AlF3などの基板上に載っているPVDFの分解プロセスを解析します。

## 解析対象
- PVDF_on_AlF3_cell_repeat_1x1x2_P0.000_T300K_HT500K.traj
- PVDF_on_Al2O3_cell_repeat_2x2x1_with_H2O_d2_0_P0.000_T300K_HT1600K.traj
- その他のPVDF-基板系のトラジェクトリファイル

## 解析内容
1. HF生成の時間変化
2. AlF₃/AlF生成の時間変化
3. F原子の基板層への貫通深度
4. 分子種の同定（CF₄, C₂F₄, CO, CO₂, H₂, H₂Oなど）
5. 反応速度の計算
6. 過剰加速判定

In [ ]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from typing import Dict, List, Tuple
from ase.io import read, Trajectory
from ase.geometry import get_distances
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# pfcc_extrasのインポート
try:
    from pfcc_extras.structure.molecule import get_mol_list
    PFCC_AVAILABLE = True
except ImportError:
    print("警告: pfcc_extrasが利用できません。一部の機能が制限されます。")
    PFCC_AVAILABLE = False

print("ライブラリのインポート完了")

## 設定パラメータ

ここで解析したいトラジェクトリファイルのパスを指定してください。

In [ ]:
# ===== 設定パラメータ =====

# 解析対象のトラジェクトリファイルパス（複数指定可能）
TRAJ_PATHS = [
    "/home/jovyan/Kaori/MD/LiB_2/structure/MD_PVDF/heat/PVDF_on_AlF3_cell_repeat_1x1x2_P0.000_T300K_HT500K.traj",
    "/home/jovyan/Kaori/MD/LiB_2/structure/MD_PVDF/heat/PVDF_on_Al2O3_cell_repeat_2x2x1_with_H2O_d2_0_P0.000_T300K_HT1600K.traj",
]

# 出力ディレクトリ
OUTPUT_DIR = "./substrate_pvdf_analysis_results"

# 解析パラメータ
ANALYSIS_PARAMS = {
    'HF_cutoff': 1.0,        # HF結合の判定距離 (Å)
    'AlF_cutoff': 2.0,       # Al-F結合の判定距離 (Å)
    'OH_cutoff': 1.1,        # O-H結合の判定距離 (Å)
    'CF_cutoff': 1.6,        # C-F結合判定 (Å)
    'CC_cutoff': 1.8,        # C-C結合判定 (Å)
    'CO_cutoff': 1.5,        # C-O結合判定 (Å)
    'HH_cutoff': 1.0,        # H-H結合判定 (Å)
    'frame_interval': 10,    # 解析するフレーム間隔（全フレーム解析は重いため）
}

# グラフのスタイル設定
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

# 出力ディレクトリの作成
Path(OUTPUT_DIR).mkdir(exist_ok=True, parents=True)
print(f"出力ディレクトリ: {OUTPUT_DIR}")

## 解析関数の定義

In [ ]:
def identify_molecules(atoms, params: Dict) -> Dict[str, float]:
    """
    原子系から分子種を同定
    
    Args:
        atoms: ASE Atoms オブジェクト
        params: 解析パラメータ
    
    Returns:
        分子種のカウント辞書
    """
    molecules = {
        'HF': 0,
        'CF4': 0,
        'C2F4': 0,
        'CO': 0,
        'CO2': 0,
        'H2': 0,
        'H2O': 0,
        'AlF3': 0,
        'Al_metal_reduced': 0,
    }
    
    positions = atoms.positions
    symbols = atoms.symbols
    
    # 原子インデックス取得
    H_idx = np.where(symbols == 'H')[0]
    F_idx = np.where(symbols == 'F')[0]
    C_idx = np.where(symbols == 'C')[0]
    Al_idx = np.where(symbols == 'Al')[0]
    O_idx = np.where(symbols == 'O')[0]
    
    # HF検出
    if len(H_idx) > 0 and len(F_idx) > 0:
        h_pos = positions[H_idx]
        f_pos = positions[F_idx]
        dists = get_distances(h_pos, f_pos, cell=atoms.cell, pbc=atoms.pbc)[1]
        molecules['HF'] = (dists < params['HF_cutoff']).sum()
    
    # CF₄検出
    for c_idx in C_idx:
        c_pos = positions[c_idx]
        if len(F_idx) > 0:
            dists = np.linalg.norm(positions[F_idx] - c_pos, axis=1)
            n_F_bonded = (dists < params['CF_cutoff']).sum()
            if n_F_bonded == 4:
                molecules['CF4'] += 1
    
    # CO検出
    if len(C_idx) > 0 and len(O_idx) > 0:
        for c_idx in C_idx:
            c_pos = positions[c_idx]
            dists = np.linalg.norm(positions[O_idx] - c_pos, axis=1)
            n_O = (dists < params['CO_cutoff']).sum()
            if n_O == 1:
                molecules['CO'] += 1
            elif n_O == 2:
                molecules['CO2'] += 0.5
    
    # CO₂検出
    if len(C_idx) > 0 and len(O_idx) >= 2:
        for c_idx in C_idx:
            c_pos = positions[c_idx]
            dists = np.linalg.norm(positions[O_idx] - c_pos, axis=1)
            n_O = (dists < params['CO_cutoff']).sum()
            if n_O == 2:
                molecules['CO2'] += 1
    
    # H₂検出
    if len(H_idx) >= 2:
        h_dists = get_distances(positions[H_idx], positions[H_idx], cell=atoms.cell, pbc=atoms.pbc)[1]
        np.fill_diagonal(h_dists, np.inf)
        molecules['H2'] = ((h_dists < params['HH_cutoff']).sum()) / 2
    
    # H₂O検出
    if len(H_idx) > 0 and len(O_idx) > 0:
        for o_idx in O_idx:
            o_pos = positions[o_idx]
            dists = np.linalg.norm(positions[H_idx] - o_pos, axis=1)
            n_H = (dists < params['OH_cutoff']).sum()
            if n_H >= 2:
                molecules['H2O'] += 1
    
    # AlF₃検出
    for al_idx in Al_idx:
        al_pos = positions[al_idx]
        if len(F_idx) > 0:
            dists = np.linalg.norm(positions[F_idx] - al_pos, axis=1)
            n_F = (dists < params['AlF_cutoff']).sum()
            if n_F >= 3:
                molecules['AlF3'] += 1
    
    return molecules


def analyze_trajectory(traj_path: str, params: Dict) -> Tuple[pd.DataFrame, List[Dict]]:
    """
    トラジェクトリファイルを解析
    
    Args:
        traj_path: trajectoryファイルのパス
        params: 解析パラメータの辞書
    
    Returns:
        解析結果のDataFrameと分子種履歴のタプル
    """
    print(f"\n=== Trajectory解析開始: {Path(traj_path).name} ===")
    
    # trajectoryの読み込み
    try:
        traj = read(traj_path, ":")
        print(f"✓ 読み込み完了: {len(traj)} フレーム")
    except FileNotFoundError:
        print(f"✗ エラー: ファイルが見つかりません: {traj_path}")
        return pd.DataFrame(), []
    except Exception as e:
        print(f"✗ エラー: {e}")
        return pd.DataFrame(), []
    
    results = []
    molecules_history = []
    frame_interval = params['frame_interval']
    
    print(f"解析フレーム間隔: {frame_interval}")
    print(f"解析対象フレーム数: {len(traj[::frame_interval])}")
    
    for i, atoms in enumerate(traj[::frame_interval]):
        frame_num = i * frame_interval
        
        # 原子インデックスの取得
        h_indices = [a.index for a in atoms if a.symbol == 'H']
        f_indices = [a.index for a in atoms if a.symbol == 'F']
        o_indices = [a.index for a in atoms if a.symbol == 'O']
        al_indices = [a.index for a in atoms if a.symbol == 'Al']
        
        # 基板層の検出（tagまたはZ座標で判定）
        al_substrate_indices = [a.index for a in atoms if a.symbol == 'Al' and hasattr(a, 'tag') and a.tag == 2]
        
        if not al_substrate_indices and al_indices:
            al_positions = atoms.positions[al_indices]
            al_z = al_positions[:, 2]
            z_threshold = np.percentile(al_z, 25)  # 下位25%を基板と仮定
            al_substrate_indices = [al_indices[j] for j, z in enumerate(al_z) if z < z_threshold]
        
        if not al_substrate_indices:
            al_substrate_indices = al_indices
        
        all_positions = atoms.positions
        
        # HF生成数
        n_hf = 0
        if h_indices and f_indices:
            h_pos = all_positions[h_indices]
            f_pos = all_positions[f_indices]
            distances_hf = get_distances(h_pos, f_pos, cell=atoms.cell, pbc=atoms.pbc)[1]
            n_hf = (distances_hf < params['HF_cutoff']).sum()
        
        # Al-F結合数
        n_alf = 0
        if al_substrate_indices and f_indices:
            al_pos = all_positions[al_substrate_indices]
            f_pos = all_positions[f_indices]
            distances_alf = get_distances(al_pos, f_pos, cell=atoms.cell, pbc=atoms.pbc)[1]
            n_alf = (distances_alf < params['AlF_cutoff']).sum()
        
        # O-H結合数
        n_oh = 0
        if o_indices and h_indices:
            o_pos = all_positions[o_indices]
            h_pos = all_positions[h_indices]
            distances_oh = get_distances(o_pos, h_pos, cell=atoms.cell, pbc=atoms.pbc)[1]
            n_oh = (distances_oh < params['OH_cutoff']).sum()
        
        # F貫通深度
        f_penetration = 0.0
        if f_indices and al_substrate_indices:
            f_z = all_positions[f_indices][:, 2]
            al_surface_z = all_positions[al_substrate_indices][:, 2].max()
            penetrating_f = f_z[f_z < al_surface_z]
            if len(penetrating_f) > 0:
                f_penetration = al_surface_z - penetrating_f.min()
        
        # 温度
        try:
            temperature = atoms.get_temperature()
        except:
            temperature = 0.0
        
        # 分子種同定
        molecules = identify_molecules(atoms, params)
        molecules_history.append(molecules)
        
        # タイムステップの計算（仮定: 0.1 fs/step）
        time_ps = frame_num * 0.0001
        
        # 結果を記録
        result_dict = {
            'frame': frame_num,
            'time_ps': time_ps,
            'temperature_K': temperature,
            'n_HF': n_hf,
            'n_AlF': n_alf,
            'n_OH': n_oh,
            'F_penetration_A': f_penetration,
        }
        
        # 分子種データを追加
        for mol_name, mol_count in molecules.items():
            result_dict[f'mol_{mol_name}'] = mol_count
        
        results.append(result_dict)
        
        # 進捗表示
        if (i + 1) % 50 == 0:
            print(f"  処理済み: {i + 1}/{len(traj[::frame_interval])} フレーム")
    
    print(f"✓ 解析完了: {len(results)} フレーム\n")
    
    return pd.DataFrame(results), molecules_history


def plot_analysis_results(df: pd.DataFrame, output_path: str, title_prefix: str = ""):
    """
    解析結果をグラフ化
    
    Args:
        df: 解析結果のDataFrame
        output_path: 出力ファイルパス
        title_prefix: タイトルのプレフィックス
    """
    fig, axes = plt.subplots(3, 3, figsize=(18, 14))
    fig.suptitle(f'{title_prefix} - 基板上PVDF分解解析', fontsize=16, fontweight='bold')
    
    # 1. HF生成
    axes[0, 0].plot(df['time_ps'], df['n_HF'], 'o-', color='#E63946', linewidth=2, markersize=3)
    axes[0, 0].set_xlabel('Time (ps)')
    axes[0, 0].set_ylabel('Number of HF')
    axes[0, 0].set_title('HF Generation', fontweight='bold')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. AlF₃生成
    axes[0, 1].plot(df['time_ps'], df['n_AlF'], 'o-', color='#457B9D', linewidth=2, markersize=3)
    axes[0, 1].set_xlabel('Time (ps)')
    axes[0, 1].set_ylabel('Number of Al-F bonds')
    axes[0, 1].set_title('AlF₃ Formation', fontweight='bold')
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. F貫通
    axes[0, 2].plot(df['time_ps'], df['F_penetration_A'], 'o-', color='#06A77D', linewidth=2, markersize=3)
    axes[0, 2].set_xlabel('Time (ps)')
    axes[0, 2].set_ylabel('Penetration Depth (Å)')
    axes[0, 2].set_title('F Penetration', fontweight='bold')
    axes[0, 2].grid(True, alpha=0.3)
    
    # 4. CF₄
    if 'mol_CF4' in df.columns:
        axes[1, 0].plot(df['time_ps'], df['mol_CF4'], 'o-', color='#F4A261', linewidth=2, markersize=3)
        axes[1, 0].set_xlabel('Time (ps)')
        axes[1, 0].set_ylabel('Number of CF₄')
        axes[1, 0].set_title('CF₄ Formation', fontweight='bold')
        axes[1, 0].grid(True, alpha=0.3)
    
    # 5. CO/CO₂
    if 'mol_CO' in df.columns and 'mol_CO2' in df.columns:
        axes[1, 1].plot(df['time_ps'], df['mol_CO'], 'o-', color='#8B4513', linewidth=2, markersize=3, label='CO')
        axes[1, 1].plot(df['time_ps'], df['mol_CO2'], 's-', color='#A0522D', linewidth=2, markersize=3, label='CO₂')
        axes[1, 1].set_xlabel('Time (ps)')
        axes[1, 1].set_ylabel('Number')
        axes[1, 1].set_title('CO/CO₂ Formation', fontweight='bold')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
    
    # 6. H₂/H₂O
    if 'mol_H2' in df.columns and 'mol_H2O' in df.columns:
        axes[1, 2].plot(df['time_ps'], df['mol_H2'], 'o-', color='#4682B4', linewidth=2, markersize=3, label='H₂')
        axes[1, 2].plot(df['time_ps'], df['mol_H2O'], 's-', color='#5F9EA0', linewidth=2, markersize=3, label='H₂O')
        axes[1, 2].set_xlabel('Time (ps)')
        axes[1, 2].set_ylabel('Number')
        axes[1, 2].set_title('H₂/H₂O Formation', fontweight='bold')
        axes[1, 2].legend()
        axes[1, 2].grid(True, alpha=0.3)
    
    # 7. 温度
    if 'temperature_K' in df.columns and df['temperature_K'].max() > 0:
        axes[2, 0].plot(df['time_ps'], df['temperature_K'], '-', color='#264653', linewidth=1.5)
        axes[2, 0].set_xlabel('Time (ps)')
        axes[2, 0].set_ylabel('Temperature (K)')
        axes[2, 0].set_title('Temperature', fontweight='bold')
        axes[2, 0].grid(True, alpha=0.3)
    
    # 8. AlF₃分子
    if 'mol_AlF3' in df.columns:
        axes[2, 1].plot(df['time_ps'], df['mol_AlF3'], 'o-', color='#264653', linewidth=2, markersize=3)
        axes[2, 1].set_xlabel('Time (ps)')
        axes[2, 1].set_ylabel('Number of AlF₃')
        axes[2, 1].set_title('AlF₃ Molecules', fontweight='bold')
        axes[2, 1].grid(True, alpha=0.3)
    
    # 9. サマリーテキスト
    axes[2, 2].axis('off')
    summary_text = f"解析サマリー\n\n"
    summary_text += f"総フレーム数: {len(df)}\n"
    summary_text += f"シミュレーション時間: {df['time_ps'].max():.2f} ps\n\n"
    
    # HF生成開始時刻
    hf_frames = df[df['n_HF'] > 0]
    if len(hf_frames) > 0:
        summary_text += f"HF生成開始: {hf_frames.iloc[0]['time_ps']:.3f} ps\n"
    else:
        summary_text += "HF生成: 未観測\n"
    
    # AlF生成開始時刻
    alf_frames = df[df['n_AlF'] > 0]
    if len(alf_frames) > 0:
        summary_text += f"AlF₃生成開始: {alf_frames.iloc[0]['time_ps']:.3f} ps\n"
    else:
        summary_text += "AlF₃生成: 未観測\n"
    
    # F貫通
    max_penetration = df['F_penetration_A'].max()
    if max_penetration > 0:
        summary_text += f"最大F貫通深度: {max_penetration:.2f} Å\n"
    else:
        summary_text += "F貫通: 未観測\n"
    
    # 最終状態
    final_row = df.iloc[-1]
    summary_text += f"\n最終状態:\n"
    summary_text += f"  HF: {final_row['n_HF']:.0f}\n"
    summary_text += f"  Al-F結合: {final_row['n_AlF']:.0f}\n"
    if 'mol_CF4' in df.columns:
        summary_text += f"  CF₄: {final_row['mol_CF4']:.0f}\n"
    
    axes[2, 2].text(0.1, 0.9, summary_text, transform=axes[2, 2].transAxes,
                   fontsize=10, verticalalignment='top', family='monospace',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))
    
    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    print(f"✓ グラフを保存: {output_path}")
    plt.close()


print("解析関数の定義完了")

## 解析の実行

In [ ]:
# 各トラジェクトリファイルを解析
all_results = {}

for traj_path in TRAJ_PATHS:
    # ファイル名の取得
    file_name = Path(traj_path).stem
    
    # 解析実行
    df, molecules_history = analyze_trajectory(traj_path, ANALYSIS_PARAMS)
    
    if df.empty:
        print(f"スキップ: {file_name}\n")
        continue
    
    # 結果を保存
    all_results[file_name] = {
        'df': df,
        'molecules_history': molecules_history
    }
    
    # CSVに保存
    csv_path = Path(OUTPUT_DIR) / f"{file_name}_analysis.csv"
    df.to_csv(csv_path, index=False)
    print(f"✓ CSVを保存: {csv_path}")
    
    # グラフ作成
    plot_path = Path(OUTPUT_DIR) / f"{file_name}_analysis.png"
    plot_analysis_results(df, str(plot_path), title_prefix=file_name)
    print()

print(f"\n=== 全ての解析が完了しました ===")
print(f"解析結果は {OUTPUT_DIR} に保存されています。")

## 複数ファイルの比較

In [ ]:
# 複数のトラジェクトリを比較
if len(all_results) > 1:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('基板種類による比較', fontsize=16, fontweight='bold')
    
    colors = ['#E63946', '#457B9D', '#06A77D', '#F4A261', '#2A9D8F']
    
    for idx, (name, data) in enumerate(all_results.items()):
        df = data['df']
        color = colors[idx % len(colors)]
        label = name.replace('PVDF_on_', '').replace('_cell_repeat_', ' ')[:30]
        
        # HF生成比較
        axes[0, 0].plot(df['time_ps'], df['n_HF'], 'o-', color=color, 
                       linewidth=2, markersize=2, label=label, alpha=0.7)
        
        # AlF生成比較
        axes[0, 1].plot(df['time_ps'], df['n_AlF'], 'o-', color=color,
                       linewidth=2, markersize=2, label=label, alpha=0.7)
        
        # F貫通比較
        axes[1, 0].plot(df['time_ps'], df['F_penetration_A'], 'o-', color=color,
                       linewidth=2, markersize=2, label=label, alpha=0.7)
        
        # 温度比較
        if 'temperature_K' in df.columns and df['temperature_K'].max() > 0:
            axes[1, 1].plot(df['time_ps'], df['temperature_K'], '-', color=color,
                           linewidth=2, label=label, alpha=0.7)
    
    axes[0, 0].set_xlabel('Time (ps)')
    axes[0, 0].set_ylabel('Number of HF')
    axes[0, 0].set_title('HF Generation Comparison')
    axes[0, 0].legend(fontsize=8)
    axes[0, 0].grid(True, alpha=0.3)
    
    axes[0, 1].set_xlabel('Time (ps)')
    axes[0, 1].set_ylabel('Number of Al-F bonds')
    axes[0, 1].set_title('AlF₃ Formation Comparison')
    axes[0, 1].legend(fontsize=8)
    axes[0, 1].grid(True, alpha=0.3)
    
    axes[1, 0].set_xlabel('Time (ps)')
    axes[1, 0].set_ylabel('Penetration Depth (Å)')
    axes[1, 0].set_title('F Penetration Comparison')
    axes[1, 0].legend(fontsize=8)
    axes[1, 0].grid(True, alpha=0.3)
    
    axes[1, 1].set_xlabel('Time (ps)')
    axes[1, 1].set_ylabel('Temperature (K)')
    axes[1, 1].set_title('Temperature Comparison')
    axes[1, 1].legend(fontsize=8)
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    comparison_path = Path(OUTPUT_DIR) / "comparison_all.png"
    plt.savefig(comparison_path, dpi=300, bbox_inches='tight')
    print(f"✓ 比較グラフを保存: {comparison_path}")
    plt.show()
else:
    print("比較するファイルが1つだけのため、比較グラフはスキップします。")

## サマリーテーブルの作成

In [ ]:
# サマリーテーブルの作成
summary_data = []

for name, data in all_results.items():
    df = data['df']
    
    # HF生成開始時刻
    hf_frames = df[df['n_HF'] > 0]
    hf_start = hf_frames.iloc[0]['time_ps'] if len(hf_frames) > 0 else np.nan
    
    # AlF生成開始時刻
    alf_frames = df[df['n_AlF'] > 0]
    alf_start = alf_frames.iloc[0]['time_ps'] if len(alf_frames) > 0 else np.nan
    
    # 最大F貫通深度
    max_penetration = df['F_penetration_A'].max()
    
    # 最終状態
    final_row = df.iloc[-1]
    
    # 最大温度
    max_temp = df['temperature_K'].max() if 'temperature_K' in df.columns else 0
    
    summary_data.append({
        'File': name,
        'Total Frames': len(df),
        'Simulation Time (ps)': df['time_ps'].max(),
        'Max Temperature (K)': max_temp,
        'HF Start (ps)': hf_start,
        'AlF3 Start (ps)': alf_start,
        'Max F Penetration (Å)': max_penetration,
        'Final HF': final_row['n_HF'],
        'Final Al-F bonds': final_row['n_AlF'],
    })

summary_df = pd.DataFrame(summary_data)

# サマリーテーブルの表示
print("\n=== 解析サマリーテーブル ===")
print(summary_df.to_string(index=False))

# CSVに保存
summary_csv_path = Path(OUTPUT_DIR) / "analysis_summary.csv"
summary_df.to_csv(summary_csv_path, index=False)
print(f"\n✓ サマリーテーブルを保存: {summary_csv_path}")

## 結論

このノートブックで、以下の解析が完了しました：

1. 各基板上のPVDF分解プロセスの詳細解析
2. HF、AlF₃などの生成物の時間変化の追跡
3. F原子の基板への貫通深度の測定
4. 分子種の同定と定量
5. 複数の基板種類の比較

解析結果は `substrate_pvdf_analysis_results/` ディレクトリに保存されています。